In [1]:
import pandas as pd
from enum import Enum
import os

from ipynb.fs.full.explainerSimilarityDynamic import gettingExplainerProperties

(False, '\n- Implementation Mismatch: This is a Sklearn model but /Tabular/DeepSHAPLocal only supports TensorFlow 1 and TensorFlow 2 implementations.\n- AI Method Mismatch: The model is a Random Forest but /Tabular/DeepSHAPLocal only supports Neural Networks (Computer).')
10.299999999999999
e2                           /Images/Anchors  /Images/Counterfactuals  \
explainer                                                               
/Images/Anchors                     1.000000                 0.509709   
/Images/Counterfactuals             0.509709                 1.000000   
/Images/GradCam                     0.432039                 0.432039   
/Images/GradCamTorch                0.432039                 0.900000   
/Images/IntegratedGradients         0.417476                 0.432039   
/Images/LIME                        0.564725                 0.580097   
/Images/NearestNeighbours           0.359763                 0.388889   
/Misc/AIModePerformance             0.349515       

[('/Tabular/IREX', 0.9), ('/Tabular/KernelSHAPGlobal', 0.6496651384394104), ('/Tabular/Importance', 0.575950647249191)]


In [2]:
PROPERTIES = {}
PROPERTIES['Explainer'] = 0
PROPERTIES['ExplainerDescription'] = 1
#PROPERTIES['ExplainabilityTechnique'] = 2
PROPERTIES['ExplainabilityTechniqueType'] = 2
PROPERTIES['DatasetType'] = 3
PROPERTIES['ExplanationOutputType'] = 4
PROPERTIES['ExplanationDescription'] = 5
PROPERTIES['Concurrentness'] = 6
PROPERTIES['Portability'] = 7
PROPERTIES['Scope'] = 8
PROPERTIES['TargetType'] = 9
PROPERTIES['OutputType'] = 10
PROPERTIES['Complexity'] = 11
PROPERTIES['AIMethodType'] = 12
PROPERTIES['AITaskType'] = 13
PROPERTIES['Backend'] = 14
PROPERTIES['metadata'] = 15

SIMPLE_PROPERTIES = [PROPERTIES['DatasetType'], PROPERTIES['Concurrentness'], PROPERTIES['Scope'], PROPERTIES['Portability'], PROPERTIES['TargetType'], PROPERTIES['Complexity']]
COMPLEX_PROPERTIES = [PROPERTIES['ExplainabilityTechniqueType'], PROPERTIES['ExplanationOutputType']]
SIMPLE_MULT_PROPERTIES = [PROPERTIES['Backend']]
COMPLEX_MULT_PROPERTIES = [PROPERTIES['OutputType'], PROPERTIES['AIMethodType'], PROPERTIES['AITaskType']]

In [3]:
formatted_df = gettingExplainerProperties()

### Natural language explanations about similarity

#### Getting similarities in natural language

In [4]:
def getRow(df, explainer):
    return df.loc[df['Explainer'] == explainer].values.flatten().tolist()

In [5]:
def getSimNL_simple_attr(attribute, e1_value, e2_value):
    return

In [6]:
def getExplanationComplex(explainer1_atts, explainer2_atts, explanation_original, explanationToInclude, isBackend=False):
    common_props = [x for x in explainer1_atts if x in explainer2_atts]
    common_props_len = len(common_props)
    if common_props_len > 0:
        if isBackend==False:
            explanation_original = explanation_original + explanationToInclude + common_props[0] + ", "
        else:
            explanation_original = explanation_original + explanationToInclude + ','.join(common_props) + ", "
    return explanation_original

In [7]:
def getExplanationComplexArray(explainer1_atts, explainer2_atts):
    attributes_list = list()
    for attrib_list1 in explainer1_atts:
        for attrib_list2 in explainer2_atts:
            common_props = [x for x in attrib_list1 if x in attrib_list2]
            common_props_len = len(common_props)
            if common_props_len > 0:
                attributes_list.append(common_props[0])
                
    return list(set(attributes_list))

In [8]:
def getSimNL(formatted_df, explainer1, explainer2):
    explanation = ""
    explainer1_atts = getRow(formatted_df, explainer1)
    explainer2_atts = getRow(formatted_df, explainer2)
    
    print(explainer1_atts)
    
    # explanation about why they are similar
    explanation = "They are similar because "
    if explainer1_atts[PROPERTIES['DatasetType']] == explainer2_atts[PROPERTIES['DatasetType']]:
        explanation = explanation + "they can be applied to the same dataset type: " + explainer2_atts[PROPERTIES['DatasetType']] + " data, "
    if explainer1_atts[PROPERTIES['Concurrentness']] == explainer2_atts[PROPERTIES['Concurrentness']]:
        explanation = explanation + "they have the same concurrentness: " + explainer2_atts[PROPERTIES['Concurrentness']] + ", "
    if explainer1_atts[PROPERTIES['Scope']] == explainer2_atts[PROPERTIES['Scope']]:
        explanation = explanation + "they have the same scope: " + explainer2_atts[PROPERTIES['Scope']] + ", "
    if explainer1_atts[PROPERTIES['Portability']] == explainer2_atts[PROPERTIES['Portability']]:
        explanation = explanation + "they have the same portability: " + explainer2_atts[PROPERTIES['Portability']] + ", "
    if explainer1_atts[PROPERTIES['TargetType']] == explainer2_atts[PROPERTIES['TargetType']]:
        explanation = explanation + "they have the same target type: " + explainer2_atts[PROPERTIES['TargetType']] + ", "
    if explainer1_atts[PROPERTIES['Complexity']] == explainer2_atts[PROPERTIES['Complexity']]:
        explanation = explanation + "they have the same computational complexity: " + explainer2_atts[PROPERTIES['Complexity']] + ", "

    # for the complex ones, if they share one in the array, show
    # if they share more than one, show the most deep (the one in the beginning of the array)
    explanationToInclude = "they are the same explainability technique type: "
    explanation = getExplanationComplex(explainer1_atts[PROPERTIES['ExplainabilityTechniqueType']], explainer2_atts[PROPERTIES['ExplainabilityTechniqueType']], explanation, explanationToInclude)
    explanationToInclude = "they show the same explanation type: "
    explanation = getExplanationComplex(explainer1_atts[PROPERTIES['ExplanationOutputType']], explainer2_atts[PROPERTIES['ExplanationOutputType']], explanation, explanationToInclude)
    explanationToInclude = "they use the same backend: "
    explanation = getExplanationComplex(explainer1_atts[PROPERTIES['Backend']], explainer2_atts[PROPERTIES['Backend']], explanation, explanationToInclude, True)


    attributes_list = getExplanationComplexArray(explainer1_atts[PROPERTIES['OutputType']], explainer2_atts[PROPERTIES['OutputType']])
    if len(attributes_list) > 0:
        explanation = explanation + "they show the explanation with the same output type: " + ','.join(attributes_list) + ", "
    
    attributes_list = getExplanationComplexArray(explainer1_atts[PROPERTIES['AIMethodType']], explainer2_atts[PROPERTIES['AIMethodType']])
    if len(attributes_list) > 0:
        explanation = explanation + "they are applicable to the same AI method type: " + ','.join(attributes_list) + ", "
    
    attributes_list = getExplanationComplexArray(explainer1_atts[PROPERTIES['AITaskType']], explainer2_atts[PROPERTIES['AITaskType']])
    if len(attributes_list) > 0:
        explanation = explanation + "they are applicable to the same AI task type: " + ','.join(attributes_list) + ", "
    
    
    return explanation

In [9]:
#print(getRow(formatted_df, "/Images/LIME"))

In [10]:
#getSimNL(formatted_df, matrix, "/Tabular/TreeSHAPGlobal", "/Tabular/ALE")

In [11]:
getSimNL(formatted_df, "/Images/LIME", "/Images/Anchors")

['/Images/LIME', "Uses LIME to identify the group of pixels that contribute the most to the predicted class.This method accepts 5 arguments: the 'id', the 'url' (optional),  the 'params' dictionary (optional) with the configuration parameters of the method, the 'instance' containing the image that will be explained as a matrix, or the 'image' file instead. These arguments are described below.", ['LIME', 'Simplification By Linear Regression', 'Simplification'], 'Image', ['Feature Influence Explanation'], 'Explanation displays the group of pixels that contribute positively (in green) and negatively (in red) to the prediction of the image class. More than one class may be displayed.', 'Post-hoc', 'Model-agnostic', 'Local', 'Prediction', [['image', 'figure', 'media']], 'Quadratic time', [['Supervised Machine Learning', 'Machine Learning']], [['Classification', 'Inductive Task']], ['PyTorch', 'Sklearn', 'TensorFlow 1', 'TensorFlow 2'], '"{\\n    \\"supportsAPI\\": true,\\n    \\"supportsB&W

'They are similar because they can be applied to the same dataset type: Image data, they have the same concurrentness: Post-hoc, they have the same scope: Local, they have the same portability: Model-agnostic, they have the same target type: Prediction, they have the same computational complexity: Quadratic time, they are the same explainability technique type: Simplification, they show the same explanation type: Feature Influence Explanation, they show the explanation with the same output type: image, they are applicable to the same AI method type: Machine Learning, they are applicable to the same AI task type: Classification, '